In [ ]:
# downloading open ai gymnasium and dedicated environment
!pip install gymnasium
!pip install 'gymnasium[atari,accept-rom-license]'
!apt-get install -y swig
!pip install gymnasium[box2d]

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
swig is already the newest version (4.0.2-1ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 19 not upgraded.


In [ ]:
# Importing all the neccessary libraries
import os
import random
import torch
import numpy as np
import torch.nn as neural_net
import torch.nn.functional as func
import torch.optim as optim
import torch.autograd as autograd
from torch.autograd import Variable
from collections import deque, namedtuple

In [ ]:
# Now building the neural network architecture

class Architecture(neural_net.Module):
  def __init__(self,state_size,action_space, seed = 42):
    super(Architecture,self).__init__()
    self.seed = torch.manual_seed(seed)
    self.connected_layer1 = neural_net.Linear(state_size,64)
    self.connected_layer2 = neural_net.Linear(64,64)
    self.connected_layer3 = neural_net.Linear(64,action_space)

  def forward(self,state):
    x = self.connected_layer1(state)
    x = func.relu(x)
    x = self.connected_layer2(x)
    x = func.relu(x)
    return self.connected_layer3(x)

In [ ]:
# Now setting up our environment for the lunar lander v2 open ai environment

import gymnasium as gym
env = gym.make('LunarLander-v2')
state_shape = env.observation_space.shape
state_size = env.observation_space.shape[0]
no_of_actions = env.action_space.n
state_shape, state_size, no_of_actions

((8,), 8, 4)

In [ ]:
# Now initialising parameters for the neural net architecture
lr = 5e-4
batch_size = 100
gamma = 0.99
replay_buffer_size = 100000
interpolation = 1e-3 # It is used for sub update for the target network and also called tau

In [ ]:
# Implementing  agents memory class (Experience replay)
class Memory(object):
  def __init__(self,capacity):
    self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    self.capacity = capacity
    self.memory = []

  def store_events(self,events):
    self.memory.append(events)
    if len(self.memory) > self.capacity:
      del self.memory[0]

  def sample_batches(self,batch_size):
    experiences = random.sample(self.memory, k = batch_size)
    states = torch.from_numpy(np.vstack([e[0] for e in experiences if e is not None])).float().to(self.device)
    actions = torch.from_numpy(np.vstack([e[1] for e in experiences if e is not None])).long().to(self.device)
    rewards = torch.from_numpy(np.vstack([e[2] for e in experiences if e is not None])).float().to(self.device)
    next_states = torch.from_numpy(np.vstack([e[3] for e in experiences if e is not None])).float().to(self.device)
    confirmations = torch.from_numpy(np.vstack([e[4] for e in experiences if e is not None]).astype(np.uint8)).float().to(self.device)
    return states, actions, rewards, next_states, confirmations


In [ ]:
# Creating the lander to prepare for training
class Lander():
  def __init__(self,state_size,action_size):
    self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    self.state_size = state_size
    self.action_size = action_size
    self.local_network = Architecture(state_size,action_size).to(self.device)
    self.main_network = Architecture(state_size,action_size).to(self.device)
    self.optimizer = optim.Adam(self.local_network.parameters(), lr=lr)
    self.memory = Memory(replay_buffer_size)
    self.time_step = 0

  def step(self, state, action, rewards, next_states, confirmations):
    self.memory.store_events((state, action, rewards, next_states, confirmations))
    self.time_step = (self.time_step + 1) % 4
    if self.time_step == 0:
      if len(self.memory.memory) > batch_size:
        experiences = self.memory.sample_batches(100)
        self.fit(experiences,gamma)

  def act(self,states,epsilon = 0.):
    state = torch.from_numpy(states).float().unsqueeze(0).to(self.device)
    self.local_network.eval()
    with torch.no_grad():
      action_values = self.local_network(state)
    self.local_network.train()
    if random.random() > epsilon:
      return np.argmax(action_values.cpu().data.numpy())
    else:
      return random.choice(np.arange(self.action_size))

  def fit(self,experiences, gamma):
    states, actions, rewards, next_states, confirmations = experiences
    q_targets_next = self.main_network(next_states).detach().max(1)[0].unsqueeze(1)
    normal_q_targets = rewards + gamma * q_targets_next * (1 - confirmations)
    predicted_actions = self.local_network(states).gather(1,actions)
    loss = func.mse_loss(normal_q_targets,predicted_actions)
    self.optimizer.zero_grad()
    loss.backward()
    self.optimizer.step()
    self.update(self.local_network,self.main_network,interpolation)

  def update(self, local, main, interpolation):
    for local_parameter, main_parameter in zip(local.parameters(), main.parameters()):
      main_parameter.data.copy_(interpolation * local_parameter.data + (1.0 - interpolation) * main_parameter.data)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [ ]:
# initializing our lander
lander = Lander(state_size, no_of_actions)

In [ ]:
# Training the model
total_episodes = 2000
tota_timesteps_per_episode = 1000
epsilon_starting  = 1.0
epsilon_ending  = 0.01
epsilon_decay  = 0.995
epsilon = epsilon_starting
scores_on_100_episodes = deque(maxlen = 100)

In [ ]:
for episode in range(1, total_episodes + 1):
  state, _ = env.reset()
  cumulative_rewards = 0
  for time in range(tota_timesteps_per_episode):
    action = lander.act(state,epsilon)
    next_state, reward, confirmation, _, _ = env.step(action)
    lander.step(state,action,reward,next_state,confirmation)
    state = next_state
    cumulative_rewards += reward
    if confirmation:
      break
  scores_on_100_episodes.append(cumulative_rewards)
  epsilon = max(epsilon_ending,epsilon_decay*epsilon)
  print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)), end = "")
  if episode % 100 == 0:
    print('\rEpisode {}\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
  if np.mean(scores_on_100_episodes) >= 250.0:
    print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(episode, np.mean(scores_on_100_episodes)))
    torch.save(lander.local_network.state_dict(), 'parameters_detail.pth')
    break


Episode 100	Average Score: 196.00
Episode 200	Average Score: 232.66
Episode 300	Average Score: 238.58
Episode 400	Average Score: 248.82


In [ ]:
# to show the video
import glob
import io
import base64
import imageio
from IPython.display import HTML, display
from gym.wrappers.monitoring.video_recorder import VideoRecorder

def show_video_of_model(lander, env_name):
    env = gym.make(env_name, render_mode='rgb_array')
    state, _ = env.reset()
    done = False
    frames = []
    while not done:
        frame = env.render()
        frames.append(frame)
        action = lander.act(state)
        state, reward, done, _, _ = env.step(action.item())
    env.close()
    imageio.mimsave('video.mp4', frames, fps=30)

show_video_of_model(lander, 'LunarLander-v2')

def show_video():
    mp4list = glob.glob('*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")

show_video()